## 5. Data Transformation

As second part of our report we will deal with the topic of engineering of variables, with the purpose of modifying variables of our observations to obtain a better predictive model. We begin by loading the code from the "EDA" notebook to continue with the study.

In [127]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt # plots
import seaborn as sns # plots
from scipy import stats as sts # normality test JB
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import Ridge, RidgeCV # Ridge
from sklearn.linear_model import Lasso, LassoCV # Lasso
from sklearn.linear_model import LogisticRegression # Log Regression
from sklearn.metrics import r2_score # R^2
from sklearn.model_selection import train_test_split # split data
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error # MSE
from sklearn.preprocessing import StandardScaler # estandarization
from sklearn import preprocessing # estandarization
from sklearn.ensemble import IsolationForest # outliers
from math import sqrt # sqrt
import itertools # aplanar arrays
import math
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC # SVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve # ROC Curve
from sklearn.datasets import make_classification
import random
random.seed(12345)
from sklearn.impute import KNNImputer

In [128]:
data1 = pd.read_csv('../data/Loan_training_set_1_4.csv', error_bad_lines=False, sep = ';')
data2 = pd.read_csv('../data/Loan_training_set_2_4.csv', error_bad_lines=False, sep = ';')
data3 = pd.read_csv('../data/Loan_training_set_3_4.csv', error_bad_lines=False, sep = ';')
data4 = pd.read_csv('../data/Loan_training_set_4_4.csv', error_bad_lines=False, sep = ';')
bigdata = pd.concat([data1, data2, data3, data4])

C:\Users\34639\anaconda3\envs\lending\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\34639\anaconda3\envs\lending\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\34639\anaconda3\envs\lending\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,19,129,130,131,134,135,136,139) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\34639\anaconda3\envs\lending\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,19,59) have mixed

In [141]:
data = bigdata

In [142]:
data = data[data['loan_status'] != 'Current']
data['paid'] = np.where((data['loan_status'] == 'Fully Paid'), 0, 1)
del data['loan_status']
del data['collection_recovery_fee']
del data['desc']
del data['funded_amnt']
del data['funded_amnt_inv']
del data['initial_list_status']
del data['issue_d'] 
del data['last_credit_pull_d']
del data['last_pymnt_amnt']
del data['last_pymnt_d']
del data['next_pymnt_d']
del data['out_prncp']
del data['out_prncp_inv']
del data['pymnt_plan']
del data['title']
del data['url']
del data['total_rec_int']
del data['total_rec_late_fee']
del data['total_rec_prncp']
del data['disbursement_method']
data.drop_duplicates()
data = data.dropna(subset=['loan_amnt'])
del data['total_pymnt']
del data['total_pymnt_inv']
data

<ipython-input-142-0e04da586779>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['paid'] = np.where((data['loan_status'] == 'Fully Paid'), 0, 1)


,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,paid
8,112038251,NaN,11575.0,36 months,7.35%,359.26,A,A4,Solutions Architect,6 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
10,112149045,NaN,7200.0,36 months,24.85%,285.70,E,E3,Pse,2 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
24,112052261,NaN,7500.0,36 months,7.35%,232.79,A,A4,Associate Director,7 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
42,111999259,NaN,10000.0,60 months,16.02%,243.29,C,C5,Biller,7 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
91,111808508,NaN,14000.0,36 months,16.02%,492.34,C,C5,cdl driver,7 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421088,36221446,NaN,11575.0,36 months,15.59%,404.61,D,D1,Manager,10+ years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,0
421089,36330911,NaN,12000.0,36 months,11.99%,398.52,B,B5,KYC Business Analyst,< 1 year,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1
421092,36271333,NaN,13000.0,60 months,15.99%,316.07,D,D2,Radiologist Technologist,5 years,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1
421093,36490806,NaN,12000.0,60 months,19.99%,317.86,E,E3,Painter,1 year,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1


In [143]:
data['revol_util'] = data.revol_util.astype(str)
data['revol_util'] = data.revol_util.str.strip('%').astype(float)

data['int_rate'] = data.int_rate.astype(str)
data['int_rate'] = data.int_rate.str.strip('%').astype(float)

In the first step we filter the variables and eliminate those that contain more than 90% of NA, since having a great amount of null values could alter the capacity of our model.

In [144]:
NaN = pd.DataFrame(round(data.isnull().sum()/len(data.index)*100,2),columns=['null_percent'])
Filtro = NaN['null_percent']>90
Nulls = NaN[Filtro]
data = data.drop(list(Nulls.index), axis=1)
#data.drop(data.tail(4).index,inplace=True)
data

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,paid
8,112038251,11575.0,36 months,7.35,359.26,A,A4,Solutions Architect,6 years,OWN,...,11.1,1.0,0.0,528172.0,100865.0,28100.0,120572.0,N,N,0
10,112149045,7200.0,36 months,24.85,285.70,E,E3,Pse,2 years,RENT,...,100.0,0.0,0.0,7600.0,5588.0,3600.0,4000.0,N,N,0
24,112052261,7500.0,36 months,7.35,232.79,A,A4,Associate Director,7 years,MORTGAGE,...,8.3,0.0,0.0,350617.0,45955.0,83700.0,32239.0,N,N,0
42,111999259,10000.0,60 months,16.02,243.29,C,C5,Biller,7 years,RENT,...,0.0,2.0,0.0,34200.0,10956.0,18800.0,5500.0,N,N,0
91,111808508,14000.0,36 months,16.02,492.34,C,C5,cdl driver,7 years,MORTGAGE,...,100.0,0.0,0.0,170591.0,27684.0,3000.0,30321.0,N,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421088,36221446,11575.0,36 months,15.59,404.61,D,D1,Manager,10+ years,RENT,...,42.9,0.0,0.0,29866.0,19798.0,27500.0,1766.0,N,N,0
421089,36330911,12000.0,36 months,11.99,398.52,B,B5,KYC Business Analyst,< 1 year,MORTGAGE,...,50.0,0.0,0.0,325484.0,112990.0,15400.0,98125.0,N,N,1
421092,36271333,13000.0,60 months,15.99,316.07,D,D2,Radiologist Technologist,5 years,RENT,...,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0,N,N,1
421093,36490806,12000.0,60 months,19.99,317.86,E,E3,Painter,1 year,RENT,...,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0,N,N,1


When eliminating the variables with but a 90% of null values, our dataset presents a total of 95 variables.

We continue to divide our dataset into categorical and numerical variables, to observe them separately. Although previously we have dealt with missing or null values, we still present percentages of these values, so we are going to substitute the NA in the numerical variables for the mean of that numerical variable and the categorical ones for the mode of that categorical variable.

In [145]:
numeric = data.select_dtypes(include = [np.number])
categorical = data.select_dtypes(include = [np.object])

In [146]:
numeric = numeric.apply(lambda x: x.replace("", np.nan))
numeric = numeric.apply(lambda x: x.fillna(x.mean()))

In [147]:
categorical = categorical.apply(lambda x: x.replace("", np.nan))
categorical = categorical.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [148]:
data = pd.concat([numeric, categorical], axis = 1)

A summary of the numerical variables is shown below:

In [149]:
numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462226 entries, 8 to 421094
Data columns (total 64 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   loan_amnt                       462226 non-null  float64
 1   int_rate                        462226 non-null  float64
 2   installment                     462226 non-null  float64
 3   annual_inc                      462226 non-null  float64
 4   dti                             462226 non-null  float64
 5   delinq_2yrs                     462226 non-null  float64
 6   fico_range_low                  462226 non-null  float64
 7   fico_range_high                 462226 non-null  float64
 8   inq_last_6mths                  462226 non-null  float64
 9   mths_since_last_delinq          462226 non-null  float64
 10  mths_since_last_record          462226 non-null  float64
 11  open_acc                        462226 non-null  float64
 12  pub_rec         

The following is a summary of the categorical variables:

In [150]:
categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462226 entries, 8 to 421094
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    462226 non-null  object
 1   term                  462226 non-null  object
 2   grade                 462226 non-null  object
 3   sub_grade             462226 non-null  object
 4   emp_title             462226 non-null  object
 5   emp_length            462226 non-null  object
 6   home_ownership        462226 non-null  object
 7   verification_status   462226 non-null  object
 8   purpose               462226 non-null  object
 9   zip_code              462226 non-null  object
 10  addr_state            462226 non-null  object
 11  earliest_cr_line      462226 non-null  object
 12  application_type      462226 non-null  object
 13  hardship_flag         462226 non-null  object
 14  debt_settlement_flag  462226 non-null  object
dtypes: object(15)
mem

In [151]:
del data['id']
del data['sub_grade']
del data['emp_title']
del data['zip_code']
del data['addr_state']
del data['debt_settlement_flag']
del data['hardship_flag']
del data['purpose']
del data['earliest_cr_line']

In [152]:
categorical = data.select_dtypes(include = [np.object])
categorical

,term,grade,emp_length,home_ownership,verification_status,application_type
8,36 months,A,6 years,OWN,Not Verified,Individual
10,36 months,E,2 years,RENT,Source Verified,Individual
24,36 months,A,7 years,MORTGAGE,Not Verified,Individual
42,60 months,C,7 years,RENT,Source Verified,Individual
91,36 months,C,7 years,MORTGAGE,Verified,Individual
...,...,...,...,...,...,...
421088,36 months,D,10+ years,RENT,Verified,Individual
421089,36 months,B,< 1 year,MORTGAGE,Verified,Individual
421092,60 months,D,5 years,RENT,Verified,Individual
421093,60 months,E,1 year,RENT,Source Verified,Individual


The options for each category variable will be shown to decide which variables to remove. Since a high number of options can negatively affect the model.

In [153]:
data.shape

(462226, 70)

After having studied the variables of our dataset, we have been left with a total of 86 variables, which will explain our model.

Once we have selected the number of categorical variables chosen for our model, we will proceed to reduce the number of possible values of those variables, in order to facilitate the study.

We decided to reduce the number of possible options within the variable "home_ownership", in order to obtain 3 unique categories that facilitate the model.

In [154]:
data['home_ownership'] = np.where((data['home_ownership'] != 'OWN') & (data['home_ownership'] != 'RENT'),
                                    'MORTAGE&OTHER', data['home_ownership'].values)
data.home_ownership.value_counts()

MORTAGE&OTHER    229281
RENT             184800
OWN               48145
Name: home_ownership, dtype: int64

Then we will transform the variable "emp_length" in order to present only two values within the variable, those customers who have 5 years or less working, and those who have more than 5 years working.

In [155]:
data['emp_length'].value_counts()

10+ years    177589
2 years       41329
< 1 year      37555
3 years       36658
1 year        29999
4 years       27056
5 years       26903
8 years       23173
7 years       22647
6 years       20918
9 years       18399
Name: emp_length, dtype: int64

In [156]:
data['emp_length'] = np.where((data['emp_length'] == '< 1 year') | 
                                       (data['emp_length'] == '1 year') |
                                       (data['emp_length'] == '2 years') | 
                                       (data['emp_length'] == '3 years') | 
                                       (data['emp_length'] == '4 years') | 
                                       (data['emp_length'] == '5 years'), 
                                       '<= 5', '6or+')

In [157]:
data['emp_length'].value_counts()

6or+    262726
<= 5    199500
Name: emp_length, dtype: int64

Next we will proceed to assign values to the variable "grade", which indicates the type of loan, having a higher numerical value that type of loan "A", while the lowest numerical value is the type "G".

In [158]:
grades = {'A': 6, 'B': 5, 'C': 4, 'D': 3, 'E': 2, 'F': 1, 'G': 0}

data['grade'] = data['grade'].replace(grades)

In [159]:
data['grade'].value_counts()

4    131409
5    123015
3     76338
6     74648
2     40309
1     12860
0      3647
Name: grade, dtype: int64

## 5.1. Creating Dummies

The function get_dummies allows to remove the first of the columns generated for each encoded characteristic to avoid the so-called collinearity (that one of the characteristics is a linear combination of the others), which makes it difficult for the algorithms to work properly. Once the "dummies variables" have been created, we would only have to replace the original characteristic in the dataset with the ones created.

In [160]:
categorical = data.select_dtypes(include = [np.object])
categorical.nunique()

term                   2
emp_length             2
home_ownership         3
verification_status    3
application_type       2
dtype: int64

In [161]:
dummies = pd.get_dummies(categorical)
dummies.head(5)

,term_ 36 months,term_ 60 months,emp_length_6or+,emp_length_<= 5,home_ownership_MORTAGE&OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App
8,1,0,1,0,0,1,0,1,0,0,1,0
10,1,0,0,1,0,0,1,0,1,0,1,0
24,1,0,1,0,1,0,0,1,0,0,1,0
42,0,1,1,0,0,0,1,0,1,0,1,0
91,1,0,1,0,1,0,0,0,0,1,1,0


In [162]:
dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462226 entries, 8 to 421094
Data columns (total 12 columns):
 #   Column                               Non-Null Count   Dtype
---  ------                               --------------   -----
 0   term_ 36 months                      462226 non-null  uint8
 1   term_ 60 months                      462226 non-null  uint8
 2   emp_length_6or+                      462226 non-null  uint8
 3   emp_length_<= 5                      462226 non-null  uint8
 4   home_ownership_MORTAGE&OTHER         462226 non-null  uint8
 5   home_ownership_OWN                   462226 non-null  uint8
 6   home_ownership_RENT                  462226 non-null  uint8
 7   verification_status_Not Verified     462226 non-null  uint8
 8   verification_status_Source Verified  462226 non-null  uint8
 9   verification_status_Verified         462226 non-null  uint8
 10  application_type_Individual          462226 non-null  uint8
 11  application_type_Joint App           46

Once the "dummies variables" are created, we would only have to delete the previous variables and replace them with the ones finally created.

In [163]:
del data['term']
del data['emp_length']
del data['home_ownership']
del data['verification_status']
del data['application_type']

In [164]:
dummies['paid'] = data['paid']
dummies['grade'] = data['grade']

We can finally observe the creation of 14 new variables that are called "dummies".

When creating these variables the index has been affected so we have proceeded to reconfigure it with the function "reset_index()"

In [165]:
dummies = dummies.reset_index(drop = True)
dummies

,term_ 36 months,term_ 60 months,emp_length_6or+,emp_length_<= 5,home_ownership_MORTAGE&OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,paid,grade
0,1,0,1,0,0,1,0,1,0,0,1,0,0,6
1,1,0,0,1,0,0,1,0,1,0,1,0,0,2
2,1,0,1,0,1,0,0,1,0,0,1,0,0,6
3,0,1,1,0,0,0,1,0,1,0,1,0,0,4
4,1,0,1,0,1,0,0,0,0,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462221,1,0,1,0,0,0,1,0,0,1,1,0,0,3
462222,1,0,0,1,1,0,0,0,0,1,1,0,1,5
462223,0,1,0,1,0,0,1,0,0,1,1,0,1,3
462224,0,1,0,1,0,0,1,0,1,0,1,0,1,2


## 5.2. Scaling

To make many Machine Learning algorithms used in Data Science work better, the input variables to the algorithm must be normalized. Normalizing means, in this case, compressing or extending the values of the variable so that they are in a defined range.

In [166]:
del numeric['paid']

In [167]:
scaling = preprocessing.RobustScaler()
scaled = scaling.fit_transform(numeric)
scaled = pd.DataFrame(scaled, columns=[numeric])

In [168]:
scaled

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,-0.085417,-0.9024,-0.066182,1.986456,-0.076735,0.0,1.000000,1.000000,0.0,-2.457580,...,0.0,0.0,-0.193182,-0.667238,1.0,0.0,2.112142,1.527048,0.721649,2.167452
1,-0.450000,1.8976,-0.293676,-0.338600,-0.968163,0.0,0.000000,0.000000,0.0,9.203062,...,0.0,-1.0,-1.977273,0.857633,0.0,0.0,-0.516837,-0.778004,-0.541237,-0.674537
2,-0.425000,-0.9024,-0.457306,1.015801,-0.392653,0.0,0.714286,0.714286,2.0,0.000000,...,0.0,0.5,0.295455,-0.715266,0.0,0.0,1.215458,0.198602,3.587629,0.013921
3,-0.216667,0.4848,-0.424834,-0.293928,-0.638367,0.0,0.142857,0.142857,0.0,0.000000,...,0.0,0.5,0.295455,-0.857633,2.0,0.0,-0.382503,-0.648135,0.242268,-0.637968
4,0.116667,0.4848,0.345384,0.225734,-0.577143,1.0,0.000000,0.000000,0.0,-4.158091,...,0.0,0.5,-0.840909,0.857633,0.0,0.0,0.306296,-0.243432,-0.572165,-0.032839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462221,-0.085417,0.4160,0.074068,-0.893905,0.746939,0.0,-0.142857,-0.142857,0.0,0.000000,...,0.0,-0.5,0.295455,-0.121784,0.0,0.0,-0.404390,-0.434219,0.690722,-0.729001
462222,-0.050000,-0.1600,0.055234,-0.045147,0.470204,1.0,0.000000,0.000000,0.0,-6.101531,...,1.0,0.5,0.102273,0.000000,0.0,0.0,1.088532,1.820390,0.067010,1.620201
462223,0.033333,0.4800,-0.199753,-0.677201,1.058776,0.0,-0.142857,-0.142857,0.0,0.000000,...,0.0,0.5,0.295455,0.000000,1.0,0.0,-0.296453,-0.086321,-0.180412,0.038300
462224,-0.050000,1.1200,-0.194217,-0.013544,0.755918,1.0,0.285714,0.285714,2.0,-2.943440,...,1.0,0.0,-0.272727,0.286449,0.0,0.0,-0.065761,0.500121,-0.226804,0.932547


In [169]:
data_final = pd.concat([scaled, dummies], axis = 1)
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462226 entries, 0 to 462225
Data columns (total 77 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   (loan_amnt,)                         462226 non-null  float64
 1   (int_rate,)                          462226 non-null  float64
 2   (installment,)                       462226 non-null  float64
 3   (annual_inc,)                        462226 non-null  float64
 4   (dti,)                               462226 non-null  float64
 5   (delinq_2yrs,)                       462226 non-null  float64
 6   (fico_range_low,)                    462226 non-null  float64
 7   (fico_range_high,)                   462226 non-null  float64
 8   (inq_last_6mths,)                    462226 non-null  float64
 9   (mths_since_last_delinq,)            462226 non-null  float64
 10  (mths_since_last_record,)            462226 non-null  float64
 11  (open_acc,)  

In [170]:
data_final

,"(loan_amnt,)","(int_rate,)","(installment,)","(annual_inc,)","(dti,)","(delinq_2yrs,)","(fico_range_low,)","(fico_range_high,)","(inq_last_6mths,)","(mths_since_last_delinq,)",...,home_ownership_MORTAGE&OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,paid,grade
0,-0.085417,-0.9024,-0.066182,1.986456,-0.076735,0.0,1.000000,1.000000,0.0,-2.457580,...,0,1,0,1,0,0,1,0,0,6
1,-0.450000,1.8976,-0.293676,-0.338600,-0.968163,0.0,0.000000,0.000000,0.0,9.203062,...,0,0,1,0,1,0,1,0,0,2
2,-0.425000,-0.9024,-0.457306,1.015801,-0.392653,0.0,0.714286,0.714286,2.0,0.000000,...,1,0,0,1,0,0,1,0,0,6
3,-0.216667,0.4848,-0.424834,-0.293928,-0.638367,0.0,0.142857,0.142857,0.0,0.000000,...,0,0,1,0,1,0,1,0,0,4
4,0.116667,0.4848,0.345384,0.225734,-0.577143,1.0,0.000000,0.000000,0.0,-4.158091,...,1,0,0,0,0,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462221,-0.085417,0.4160,0.074068,-0.893905,0.746939,0.0,-0.142857,-0.142857,0.0,0.000000,...,0,0,1,0,0,1,1,0,0,3
462222,-0.050000,-0.1600,0.055234,-0.045147,0.470204,1.0,0.000000,0.000000,0.0,-6.101531,...,1,0,0,0,0,1,1,0,1,5
462223,0.033333,0.4800,-0.199753,-0.677201,1.058776,0.0,-0.142857,-0.142857,0.0,0.000000,...,0,0,1,0,0,1,1,0,1,3
462224,-0.050000,1.1200,-0.194217,-0.013544,0.755918,1.0,0.285714,0.285714,2.0,-2.943440,...,0,0,1,0,1,0,1,0,1,2


We found the final dataset with 79 variables, these variables will be studied later by the Lasso to determine which of them are more important.

## 5.3. LASSO

In Statistics and Machine Learning, Lasso (least absolute shrinkage and selection operator), is a regression analysis method that performs variable selection and regularization to improve the accuracy and interpretability of the statistical model produced by it.

In [171]:
no_dummies = ['term_ 36 months', 'emp_length_<= 5 years', 'home_ownership_OTHER', 'verification_status_Not Verified', 'application_type_Individual']

In [172]:
endogena = ['paid'] 

exogenas = list(set(list(data_final.columns))-set(endogena)-set(list(no_dummies))) 

X = data_final[exogenas].values
Y = data_final[endogena].values

C:\Users\34639\anaconda3\envs\lending\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [173]:
%%time
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 200, normalize = True)
lassocv.fit(X, Y)
lassocv.alpha_

model_lasso = Lasso(alpha=lassocv.alpha_) # lasso function
model_lasso.fit(X, Y)

C:\Users\34639\anaconda3\envs\lending\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Wall time: 2min 23s


C:\Users\34639\anaconda3\envs\lending\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18798.55550993899, tolerance: 8.726302260799239
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=2.7323366845319208e-06)

Below is a list with the coefficients of all variables in both positive and negative values

In [174]:
lasso_coefficients = pd.DataFrame(model_lasso.coef_, exogenas, columns = ['Coefficients'])
lasso_coefficients.loc['Intercept'] = model_lasso.intercept_
lasso_coefficients

,Coefficients
"(mths_since_last_major_derog,)",0.000105
"(open_acc,)",0.000548
"(tot_hi_cred_lim,)",0.004101
"(mort_acc,)",-0.004722
grade,-0.009282
...,...
"(acc_open_past_24mths,)",-0.000219
"(inq_last_6mths,)",-0.003858
"(pub_rec_bankruptcies,)",-0.017126
"(num_rev_tl_bal_gt_0,)",-0.007623


In [175]:
lasso_coefficients.sort_values(['Coefficients'], ascending = False).head(30)

,Coefficients
application_type_Joint App,0.240777
term_ 60 months,0.143847
Intercept,0.132000
"(installment,)",0.105605
home_ownership_RENT,0.035142
"(fico_range_low,)",0.029299
home_ownership_OWN,0.026115
"(dti,)",0.025794
"(acc_now_delinq,)",0.023561
"(num_rev_accts,)",0.021819


In [176]:
lasso_coefficients.sort_values(['Coefficients']).head(20)

,Coefficients
"(last_fico_range_high,)",-0.381087
"(loan_amnt,)",-0.106536
"(int_rate,)",-0.036708
"(total_acc,)",-0.031584
"(pub_rec_bankruptcies,)",-0.017126
"(tot_cur_bal,)",-0.013938
"(total_il_high_credit_limit,)",-0.013567
"(num_tl_30dpd,)",-0.011477
"(num_op_rev_tl,)",-0.009286
grade,-0.009282


In [188]:
important_variables = lasso_coefficients[(lasso_coefficients['Coefficients'] > 0.01) | (lasso_coefficients['Coefficients'] < -0.01)]
a=important_variables.sort_values(['Coefficients'], ascending = False)
a.Coefficients

application_type_Joint App       0.240777
term_ 60 months                  0.143847
Intercept                        0.132000
(installment,)                   0.105605
home_ownership_RENT              0.035142
(fico_range_low,)                0.029299
home_ownership_OWN               0.026115
(dti,)                           0.025794
(acc_now_delinq,)                0.023561
(num_rev_accts,)                 0.021819
home_ownership_MORTAGE&OTHER     0.017984
(num_il_tl,)                     0.013238
(mo_sin_old_rev_tl_op,)          0.013138
verification_status_Verified     0.011736
(num_sats,)                      0.011127
(num_tl_30dpd,)                 -0.011477
(total_il_high_credit_limit,)   -0.013567
(tot_cur_bal,)                  -0.013938
(pub_rec_bankruptcies,)         -0.017126
(total_acc,)                    -0.031584
(int_rate,)                     -0.036708
(loan_amnt,)                    -0.106536
(last_fico_range_high,)         -0.381087
Name: Coefficients, dtype: float64

We finally decided to stay with variables whose coefficient was greater than 0.01 both positive and negative.

In [190]:
final_df = data_final.loc[:,['application_type_Joint App',
 ('installment',),
 'term_ 60 months',
 ('dti',),
 ('fico_range_low',),
 ('acc_now_delinq',),
 ('num_rev_accts',),  
 ('num_il_tl',), 
 ('mo_sin_old_rev_tl_op',),                           
 ('num_sats',),
 'verification_status_Verified',
 ('num_tl_30dpd',),
 ('pub_rec_bankruptcies',),
 ('total_il_high_credit_limit',),
 ('int_rate',),
 ('total_acc',),
 ('loan_amnt',),
 'home_ownership_RENT',
 'home_ownership_OWN',
 'home_ownership_MORTAGE&OTHER',
 ('last_fico_range_high',),
 ('tot_cur_bal',)]]

C:\Users\34639\anaconda3\envs\lending\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [191]:
final_ind = ['application_type_Joint App',
 ('installment',),
 'term_ 60 months',
 ('dti',),
 ('fico_range_low',),
 ('acc_now_delinq',),
 ('num_rev_accts',),  
 ('num_il_tl',), 
 ('mo_sin_old_rev_tl_op',),                           
 ('num_sats',),
 'verification_status_Verified',
 ('num_tl_30dpd',),
 ('pub_rec_bankruptcies',),
 ('total_il_high_credit_limit',),
 ('int_rate',),
 ('total_acc',),
 ('loan_amnt',),
 'home_ownership_RENT',
 'home_ownership_OWN',
 'home_ownership_MORTAGE&OTHER',
 ('last_fico_range_high',),
 ('tot_cur_bal',)]

In [192]:
final_df['paid'] = data_final['paid']
final_df

,application_type_Joint App,"(installment,)",term_ 60 months,"(dti,)","(fico_range_low,)","(acc_now_delinq,)","(num_rev_accts,)","(num_il_tl,)","(mo_sin_old_rev_tl_op,)","(num_sats,)",...,"(total_il_high_credit_limit,)","(int_rate,)","(total_acc,)","(loan_amnt,)",home_ownership_RENT,home_ownership_OWN,home_ownership_MORTAGE&OTHER,"(last_fico_range_high,)","(tot_cur_bal,)",paid
0,0,-0.066182,0,-0.076735,1.000000,0.0,1.8,0.625,0.982143,1.500000,...,2.167452,-0.9024,1.3750,-0.085417,0,1,0,0.318182,2.045887,0
1,0,-0.293676,0,-0.968163,0.000000,0.0,-1.1,-0.750,0.062500,-1.166667,...,-0.674537,1.8976,-1.1875,-0.450000,1,0,0,-0.181818,-0.416741,0
2,0,-0.457306,0,-0.392653,0.714286,0.0,0.1,0.125,-1.017857,1.333333,...,0.013921,-0.9024,0.1875,-0.425000,0,0,1,0.272727,1.109212,0
3,0,-0.424834,1,-0.638367,0.142857,0.0,0.3,-0.375,-0.991071,0.666667,...,-0.637968,0.4848,-0.1875,-0.216667,1,0,0,-0.272727,-0.386501,0
4,0,0.345384,0,-0.577143,0.000000,0.0,-0.8,-0.500,-0.607143,-1.166667,...,-0.032839,0.4848,-0.8750,0.116667,0,0,1,-0.045455,0.270862,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462221,0,0.074068,0,0.746939,-0.142857,0.0,-0.1,-0.250,0.169643,-0.333333,...,-0.729001,0.4160,-0.3750,-0.085417,1,0,0,0.045455,-0.336689,0
462222,0,0.055234,0,0.470204,0.000000,0.0,0.6,4.125,0.607143,0.333333,...,1.620201,-0.1600,2.3125,-0.050000,0,0,1,-0.545455,0.886921,1
462223,0,-0.199753,1,1.058776,-0.142857,0.0,-0.4,0.625,-0.508929,-0.333333,...,0.038300,0.4800,-0.1250,0.033333,1,0,0,-1.090909,-0.255680,1
462224,0,-0.194217,1,0.755918,0.285714,0.0,0.2,-0.500,-0.732143,1.000000,...,0.932547,1.1200,-0.2500,-0.050000,1,0,0,-1.727273,-0.119125,1


We can see how we finally have 23 variables which will serve to predict the models.

In [193]:
final_df.to_csv('../data/modeldata.csv')